<a href="https://colab.research.google.com/github/peremartra/LLMOptCost/blob/main/11/11_GradioInterface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q gradio
!pip install -q transformers
!pip install -q bitsandbytes
!pip install -q accelerate


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 440.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/71.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/1

In [2]:
import gradio as gr
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
import time

In [3]:
#OPT2="memory"
#OPT2="inference"
OPT2="none"

In [4]:
# Load the base model and quantized model from Hugging Face
base_model_name = "bigscience/bloomz-560m"
custom_model_name = "oopere/bloomz-560m-quantized_"+OPT2
custom_model_name = "oopere/bloomz-560m-pruned" #Use with OPT="none" or with opt="inference" to quantize the pruned model

In [5]:
# Load the tokenizer (both models use the same tokenizer)
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

tokenizer_config.json:   0%|          | 0.00/222 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device_map = {"": device} if device != "cpu" else {}

In [7]:
# Load the base model
base_model = AutoModelForCausalLM.from_pretrained(base_model_name,
                                                  device_map=device,
                                                  )


config.json:   0%|          | 0.00/715 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

In [8]:
def return_bnb(goal="memory"):
  if goal == "memory":
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,  # Optimize for memory using 4-bit quantization
            bnb_4bit_use_double_quant=True,  # Save memory but slower on inference
            bnb_4bit_quant_type="nf4",  # Use nf4 quantization for better memory usage
            bnb_4bit_compute_dtype=torch.float16  # Depending on GPU we can change for bfloat16.
        )
  elif goal == "inference":
        bnb_config = BitsAndBytesConfig(
            load_in_8bit=True,  # Optimize for inference using 8-bit quantization
            bnb_4bit_compute_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
        )
  else:
      bnb_config=None
  return bnb_config

In [9]:
bnb_config = return_bnb(OPT2)

In [10]:
# Load the quantized model
quantized_model = AutoModelForCausalLM.from_pretrained(custom_model_name,
                    quantization_config=bnb_config,
                    device_map=device_map,
                    use_cache = False)

config.json:   0%|          | 0.00/807 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [11]:
base_model.get_memory_footprint()

2236858368

In [12]:
quantized_model.get_memory_footprint()

2236858368

In [18]:
# Define the function that will take input and pass it to both models
def compare_models(prompt):


    inputs = tokenizer(prompt, return_tensors="pt").to(base_model.device)
    # Generate output from base
    start_time_base = time.time()
    base_output = base_model.generate(**inputs)
    base_response = tokenizer.decode(base_output[0], skip_special_tokens=True)
    base_time = time.time() - start_time_base

    # Generate output from custom model
    start_time_quantized = time.time()
    quantized_output = quantized_model.generate(**inputs)
    quantized_response = tokenizer.decode(quantized_output[0], skip_special_tokens=True)
    quantized_time = time.time() - start_time_quantized

    # Generate output from custom with dynamic input truncation model
    inputs = tokenizer(prompt, return_tensors="pt", padding=False).to(base_model.device)
    start_time_dynamic = time.time()
    quantized_output = quantized_model.generate(**inputs,
                                                do_sample=False,
                                                early_stopping=True)
    quantized_response_dynamic = tokenizer.decode(quantized_output[0], skip_special_tokens=True)
    quantized_time_dynamic = time.time() - start_time_dynamic


    # Return both responses
    return base_response, quantized_response, quantized_response_dynamic, f"{base_time:.3f} seconds", f"{quantized_time:.3f} seconds", f"{quantized_time_dynamic:.3f} seconds"

In [19]:
# Create Gradio interface with additional time outputs
interface = gr.Interface(
    fn=compare_models,
    inputs=gr.Textbox(label="Input Prompt"),
    outputs=[
        gr.Textbox(label="Base Model Response"),
        gr.Textbox(label="Custom Model Response"),
        gr.Textbox(label="Dynamic Model Response"),
        gr.Textbox(label="Base Model Response Time"),
        gr.Textbox(label="Custom Model Response Time"),
        gr.Textbox(label="Dynamic Model Response"),
    ],
    title="Compare Base Model vs custom Model",
    description="Input a prompt and see how the base and quantized models respond, along with their response times."
)

In [15]:
# Launch the Gradio interface
interface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://da4e41c3cece1a0f0c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
